# Reproject Raster Script

This Python script allows for **reprojecting raster data**. Supporting various resampling methods, compression options, and tiling settings. The output raster will retain the same data type as the input raster.

## Features
- **Batch Processing:** Handles all raster files (.tif, .tiff) in the input folder.
- **Customizable Resampling Methods:** Choose from various resampling techniques depending on your use case.
- **Compression Options:** Supports `DEFLATE`, `LZW`, or no compression.
- **Tiling:** Enables tiled output for better performance on large datasets.
- **Preserves Data Type:** Ensures the output data type matches the input.

## Script Parameters
| Parameter           | Type      | Description                                                                                       |
|---------------------|-----------|---------------------------------------------------------------------------------------------------|
| `input_folder`      | `str`     | Path to the input raster folder.                                                |
| `output_fodler`     | `str`     | Path to save the output raster files.                            |
| `target_crs`        | `str`     | Target CRS in EPSG format.                                         |
| `resampling_method` | `str`     | Resampling method to use. See **Resampling Methods** section below. Default is `'bilinear'`.     |
| `compression`       | `str`     | Compression type for the output raster. Options: `'DEFLATE'`, `'LZW'`, `'NONE'`. Default is `'DEFLATE'`. |
| `tile_size`         | `tuple`   | Tile size for output raster (e.g., `(256, 256)`). Default is `(256, 256)`.                       |

## Author
- **Rubén Crespo Ceballos**

In [ ]:
from osgeo import gdal
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    """
    Create a folder if it doesn't exist.

    Parameters:
    folder_path (str): The path of the folder to be created.
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created at: {folder_path}")
    else:
        print(f"Folder already exists at: {folder_path}")

def get_raster_file_list(path):
    """
    Get a list of the raster files inside the folder
    Parameters:
    - path (str): path of the folder with the resources.

    Returns:
    - File_list (list). list of the resources.
    """
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        # "32628" is just to get here necessary ones
        if file.endswith(".tif") or file.endswith(".tiff"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def reproject_raster(input_raster, output_folder, target_crs, resampling_method='bilinear', compression='DEFLATE', tile_size=(256, 256)):
    """
    Reprojects a raster from its current CRS to the target CRS using GDAL, with various customization options.

    Parameters:
    - input_raster: str, path to the input raster file (e.g., "input.tif").
    - output_raster: str, path to the output raster file (e.g., "output_reprojected.tif").
    - target_crs: str, the target CRS in EPSG format (e.g., "EPSG:4326" for WGS 84).
    - resampling_method: str, resampling method ('nearest', 'bilinear', 'cubic', etc.), default is 'bilinear'.
    - compression: str, compression method for the output raster (e.g., 'DEFLATE', 'LZW', 'NONE'). Default is 'DEFLATE'.
    - tile_size: tuple, tile size for the output raster (e.g., (256, 256)). Default is (256, 256).
    """
    # Open the input raster
    src_ds = gdal.Open(input_raster)
    
    if src_ds is None:
        print("Error: Could not open input raster.")
        return
    
    # Get the source CRS and geotransform
    src_srs = src_ds.GetProjection()
    geotransform = src_ds.GetGeoTransform()
    x_size = src_ds.RasterXSize
    y_size = src_ds.RasterYSize

    # Get the data type of the input raster
    input_data_type = src_ds.GetRasterBand(1).DataType  # Retrieve the data type of the first band


    # Define resampling methods
    resampling_methods = {
        'nearest': gdal.GRA_NearestNeighbour,
        'bilinear': gdal.GRA_Bilinear,
        'cubic': gdal.GRA_Cubic,
        'cubic_spline': gdal.GRA_CubicSpline,
        'lanczos': gdal.GRA_Lanczos,
        'average': gdal.GRA_Average,
        'mode': gdal.GRA_Mode
    }

    # Check if the resampling method is valid
    if resampling_method not in resampling_methods:
        print(f"Error: Invalid resampling method '{resampling_method}'. Using 'bilinear' by default.")
        resampling_method = 'bilinear'

    # Get the resampling method constant
    resample_method = resampling_methods[resampling_method]

    # Create the output raster with compression and tiling
    driver = gdal.GetDriverByName('GTiff')  # GeoTIFF format
    if driver is None:
        print("Error: Could not get the GDAL driver for GeoTIFF.")
        return

    # Get the output raster dimensions and geotransform
    geotransform = src_ds.GetGeoTransform()
    x_size = src_ds.RasterXSize
    y_size = src_ds.RasterYSize

    # Create output dataset with compression, tiling, and specified CRS
    options = [
        'TILED=YES',
        f'TILESIZE={tile_size[0]},{tile_size[1]}',
        f'COMPRESS={compression}'
    ]

    # Create path for output raster
    output_raster = os.path.join(output_folder, os.path.splitext(os.path.basename(input_raster))[0]+ "_reprojected.tif")

    dst_ds = driver.Create(
        output_raster,
        x_size,
        y_size,
        src_ds.RasterCount,
        input_data_type,
        options
    )
    
    
    if dst_ds is None:
        print("Error: Could not create output raster.")
        return

    # Set the target CRS and geo-transform to the output raster
    dst_ds.SetProjection(target_crs)
    dst_ds.SetGeoTransform(geotransform)

    # Reproject the image from the input CRS to the target CRS
    gdal.ReprojectImage(src_ds, dst_ds, src_srs, target_crs, resample_method)

    print(f"Reprojection complete. Output saved to {output_raster}")

    # Close the datasets
    src_ds = None
    dst_ds = None


In [ ]:
"""Input parameters"""
input_folder = '\input_foler_path'  # Replace with your input raster folder.
output_folder = input_folder + '\output_folder'  # This is the standard.
target_crs = 'EPSG:4326'  

In [ ]:
"""Working functions"""
create_folder_if_not_exists(output_folder)
raster_list = get_raster_file_list(input_folder)

for raster_file in raster_list:
# Call the reproject_raster function with customizable parameters
    reproject_raster(
        raster_file,
        output_folder,
        target_crs,
        resampling_method='bilinear',  # Choose resampling method: 'nearest', 'bilinear', 'cubic', etc.
        compression='DEFLATE',  # Choose compression method: 'DEFLATE', 'LZW', 'NONE'
        tile_size=(256, 256))  # Set the tile size for output

### Resampling Methods
| Method          | GDAL Constant              | Description                                                                                  | Use Case                                                                                              |
|------------------|----------------------------|----------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| `nearest`        | `GRA_NearestNeighbour`     | Nearest neighbor resampling. Fastest method, no interpolation.                               | Use for categorical data like land cover or classifications.                                          |
| `bilinear`       | `GRA_Bilinear`             | Bilinear interpolation using the nearest 4 pixels.                                           | Best for continuous data like elevation or temperature where smoother results are needed.             |
| `cubic`          | `GRA_Cubic`                | Cubic convolution interpolation using the nearest 16 pixels.                                 | Provides smoother results for continuous data but slower than bilinear.                               |
| `cubic_spline`   | `GRA_CubicSpline`          | Spline interpolation for high-quality smoothing.                                             | Use when high precision and smoothness are critical, but it is computationally expensive.             |
| `lanczos`        | `GRA_Lanczos`              | Lanczos resampling using a sinc function.                                                    | Best for high-quality resampling of continuous data. Slower but produces sharp, detailed results.     |
| `average`        | `GRA_Average`              | Computes the average of all non-NODATA contributing pixels.                                  | Use for downscaling continuous data to larger pixel sizes while preserving overall trends.            |
| `mode`           | `GRA_Mode`                 | Selects the most frequently occurring value from all non-NODATA contributing pixels.         | Use for categorical data when downscaling to maintain the dominant class in aggregated areas.         |